In [ ]:
import pandas as pd
import numpy as np
import os
from PIL import Image

In [ ]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
unique_labels = [
    'agriculture', 
    'artisinal_mine', 
    'bare_ground', 
    'blooming', 
    'blow_down', 
    'clear', 
    'cloudy', 
    'conventional_mine', 
    'cultivation', 
    'habitation', 
    'haze', 
    'partly_cloudy', 
    'primary', 
    'road', 
    'selective_logging', 
    'slash_burn', 
    'water'
]

In [ ]:
def oneHotEncode(labels_list):
    onehot = [];
    for label in unique_labels: 
        onehot.append(int(label in labels_list))
    return onehot    

def oneHotDecode(labels_list):
    labels = []
    for i, probability in enumerate(labels_list):
        if probability.item() > 0.7:
            labels.append(unique_labels[i])
    return labels

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.pool = nn.MaxPool2d(2, 2)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=2, stride=1, padding='same')
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=2, stride=1, padding='same')
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=2, stride=1, padding='same')
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=2, stride=1, padding='same')
        self.fc1 = nn.Linear(256 * 8 * 8, 1000)
        self.fc2 = nn.Linear(1000, 500)
        self.fc3 = nn.Linear(500, len(unique_labels))
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Model()
model.to(device)

criterion = nn.BCEWithLogitsLoss (weight=None, reduce=True)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
model = Model()
model.load_state_dict(torch.load('model/tensor.pth'))
model.to(device)
model.eval()

In [ ]:
def predict(image_path):
    
    transform = transforms.Compose(
        [
            transforms.Resize((128, 128)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomRotation(degrees=45),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    )
    
    image = Image.open(image_path).convert("RGB")
    
    # Convert to tensor
    inputs = transform(image).to('cuda').unsqueeze(0)
    
    #predict 
    outputs = model(inputs)
    outputs = torch.sigmoid(outputs)
    return oneHotDecode(outputs[0])


In [ ]:
prediction = predict('/home/cristine/Imagens/mato_grosso_tes.jpg')
print(prediction)

In [ ]:
def validateX(x, width):
    if(x-500 < 0):
        return 500
    if(x+500 > width):
        return width
    return x

def validateY(y, height):
    if(y-300 < 0):
        return 300
    if(y+300 > height):
        return height
    return y

In [ ]:
import cv2
def print_image(x, y):
    im = PIL.Image.open('/home/cristine/Imagens/mato_grosso.jpg')
    width, height = im.size
    x = validateX(x, width)
    y = validateY(y, height)
    img = cv2.imread("/home/cristine/Imagens/mato_grosso.jpg")
    crop_img = img[y-300:y+300, x-500:x+500]
    cv2.imshow("cropped", crop_img)
    cv2.waitKey(0)

In [ ]:
import tkinter
from tkinter import *
import time
import PIL.Image, PIL.ImageTk

# Create a window
window = tkinter.Tk()
window.geometry('1000x600')
window.maxsize(1000, 600)

frame = tkinter.Frame(window, width=385, height=460, relief='raised') # relief=SUNKEN)


frame.place(x=0, y=116, anchor="nw", width=600, height=330)


frame.grid_rowconfigure(0, weight=1)
frame.grid_columnconfigure(0, weight=1)

xscrollbar = tkinter.Scrollbar(frame, orient=tkinter.HORIZONTAL)
xscrollbar.grid(row=1, column=0, sticky=tkinter.E+tkinter.W)

yscrollbar = tkinter.Scrollbar(frame)
yscrollbar.grid(row=0, column=1, sticky=tkinter.N+tkinter.S)

canvas = tkinter.Canvas(frame, bd=0, xscrollcommand=xscrollbar.set, yscrollcommand=yscrollbar.set, 
                        width=1000, height=600)
canvas.grid(row=0, column=0, sticky=tkinter.N+tkinter.S+tkinter.E+tkinter.W)

File = "/home/cristine/Imagens/mato_grosso.jpg"
img = PIL.ImageTk.PhotoImage(PIL.Image.open(File))

canvas.create_image(0,0,image=img, anchor="nw")

canvas.config(scrollregion=canvas.bbox(tkinter.ALL))

xscrollbar.config(command=canvas.xview)
yscrollbar.config(command=canvas.yview)

def on_button_pressed(event):
    print_image(event.x, event.y)

canvas.bind("<Button-1>", on_button_pressed)


time_lbl = Label(
    window,
    text=time.strftime( "%d/%m/%Y %A %H:%M:%S"),
    font=(21),
    padx=10,
    pady=5,
    bg='#d9d8d7'
    )

time_lbl.pack(expand=True)
frame.pack()
window.update()


while True:
    time_text=time.strftime("%d/%m/%Y %A %H:%M:%S")
    time_lbl.config(text=time_text)
    
    window.update()
    
#window.mainloop()